# 1. Data Loading and Initial Inspection

Before running this notebook, make sure that the dataset is present in the '/data' folder.
You can either download it directly from the link in the repo or use 'make data' to pull it down using the Kaggle API.

Now, Let's load the dataset and perform some basic inspections.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('~/Capstone/data/airbnb-listings.csv', sep=";", low_memory=False)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

### a. Check for missing values

Missing values can significantly affect the performance of your model, so it's important to know which columns have missing data

In [ ]:
# Set Pandas to display all columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.width', None)  # Ensure it doesn't wrap text

# Calculate missing values in each column and sort
missing_data = df.isnull().sum().sort_values(ascending=False)

# Display columns with missing data (if missing > 0)
print(missing_data[missing_data > 0])


# Description of Missing Data in the Dataset
The dataset contains 494,954 entries and 89 columns. Each column represents a different attribute of the Airbnb listings, and the dataset shows a varying degree of missing data across these columns. Here's a breakdown of the missing data, which reveals some important insights about the completeness of the dataset and areas where the information might be sparse.

#### Key Observations:
#### 1. Columns with High Missing Data Counts:
Reviews per Month: Missing 122,007 values, showing that reviews data is also sparsely available for some listings.
First Review and Last Review: Missing 122,007 and 121,899 values respectively, indicating that reviews may not be present for all listings, especially newer ones or those with fewer interactions.
Neighbourhood: Missing 149,551 values, which could indicate that neighborhood data is not always available for listings.
Host About: This column is missing 198,677 values, suggesting that many host profiles do not include detailed descriptions about the host.
Jurisdiction Names: Missing 360,401 values, which may indicate that jurisdiction data is missing for many listings.
Neighbourhood Group Cleansed: With 392,791 missing values, it indicates that the neighborhood grouping data is only available for a subset of listings.
Weekly Price and Monthly Price: Both columns have 397,207 and 398,863 missing values, respectively, which could imply that many listings do not have these pricing details.
Host Acceptance Rate: This column has a significant 452,696 missing values, meaning a large portion of the data for this feature is unavailable. This could be due to hosts not setting or reporting their acceptance rate, or it may not be available for listings that are not accepting guests.
License: Missing 480,358 values, which suggests that most listings don’t have a license recorded in the dataset.
Has Availability: Missing 485,647 values, which might imply that availability data is available for only a small fraction of listings.

#### 2. Columns with Moderate Missing Data:
Description: Missing 160 values, pointing to incomplete descriptions for certain listings.
Name: Missing 418 values, suggesting that a small number of listings lack a name.
Host Location: Missing 2,263 values, which suggests that location information about the host is not always provided.
Amenities: Missing 4,488 values, meaning that the feature list for some listings is incomplete.
Price: Missing 7,958 values, which is a relatively small portion compared to the overall dataset, but still significant enough to warrant attention when analyzing pricing.
Summary: Missing 16,201 values, indicating that a reasonable portion of the listings do not have a description or summary provided.

#### 3. Columns with Low Missing Data:
Last Scraped: This column has only 1 missing value, suggesting that the date for the most recent scrape is almost always recorded, and there is only one instance where it was missed. 
Room Type: Missing only 1 value, suggesting it's almost always recorded for listings. 
Geolocation: Missing 1 value, but it is still generally available for almost all listings. Street: Missing 1 value, which likely corresponds to a listing with incomplete address information.
Latitude and Longitude: Both of these geographical columns are missing only 1 value, which indicates that most listings have geographical coordinates captured.
Cancellation Policy: Missing 2 values, indicating that the cancellation policy is recorded for almost all listings.
Smart Location, Country Code, and Country: These columns have 2 or 3 missing values, respectively, suggesting that geographical data is mostly available. 



# Summary and Impact of Missing Data:
- **High Missing Data Columns**: In cases where missing data is very high, we will drop columns.

- **Columns with Moderate and Low Missing Data**: In cases where missing data is low to moderate, we may decide to impute using reasonable assumptions or drop the rows on a case by case basis.

#### Conclusion:
This dataset contains a mix of columns with minimal missing data and columns with substantial missing data. By understanding the missing data distribution, we can make informed decisions on whether to impute or remove the missing values depending on their impact on our models. Handling missing data effectively will improve the accuracy and reliability of said models.

### b. Check for duplicates

In [ ]:
# Check for duplicates in the data
duplicates = df.duplicated().sum()

print(f"Number of duplicate rows: {duplicates}")


#### Description of Duplicate Rows:
The dataset contains 0 duplicate rows, meaning that each entry in the dataset is unique. This is a positive outcome, as it ensures that the data does not include redundant or repeated records, which could potentially distort analysis or model performance. Having no duplicate rows helps maintain the integrity of the data, ensuring that each entry represents a distinct Airbnb listing without any repetitions.

### c. Check for data freshness

In [ ]:
# Convert the 'Last Scraped' column to datetime format, coerce errors to NaT
df['Last Scraped'] = pd.to_datetime(df['Last Scraped'], errors='coerce')

# Get the earliest scrape date (min) and the most recent scrape date (max)
earliest_scrape = df['Last Scraped'].min()
recent_scrape = df['Last Scraped'].max()

# Print the earliest and most recent scrape dates
print(f"The earliest scrape date was: {earliest_scrape}")
print(f"The most recent scrape date was: {recent_scrape}")


### Data Freshness Explaination
Although the dataset spans from 2015 to 2017, it still holds significant value for analysis, particularly when exploring long-term trends, market evolution, and predictive modeling. Despite its age, this data can provide essential insights into the Airbnb market, offering a foundation for understanding key variables like pricing patterns, neighborhood popularity, and host behaviors. Additionally, the dataset's comprehensive nature makes it a valuable resource, especially when newer datasets may be less complete or accessible. This dataset serves as a useful baseline for comparing past and present market conditions, allowing for the identification of growth areas and the development of strategies that remain relevant today. By leveraging this historical data, we can uncover lasting insights that inform both current analysis and future predictions.

#### 1. Revealing Long-Term Trends:
Insight into Historical Patterns: Despite the dataset being from 2015 to 2017, it offers valuable insights into long-term trends. Understanding past pricing patterns, neighborhood popularity, and host behaviors can provide a foundation for predicting future trends, particularly in a market that has reached a certain level of stability.
Market Evolution: Analyzing the data from 2015-2017 allows you to uncover shifts in customer preferences, pricing fluctuations, and new amenities that may still apply today.
#### 2. Baseline for Comparison and Predictive Modeling:
Comparing with Newer Data: This dataset serves as an essential baseline for comparing more current data, helping you track how the market has evolved. It enables the measurement of growth in features, customer satisfaction, and price shifts.
Predictive Insights: Historical data can still be valuable for predictive modeling, with trends from 2015-2017 offering insights into future market behavior, especially when combined with more recent data.
#### 3. Data Completeness and Quality:
Comprehensive Dataset: Despite its age, the dataset is relatively complete, containing diverse features like pricing, host information, and reviews. This makes it a rich data source that may be more comprehensive than newer datasets, which could be more fragmented.
Data Availability: Finding a dataset that is both up-to-date and complete can be challenging. If more recent datasets are missing key features, this dataset provides a solid, all-encompassing foundation for analysis.
#### 4. Understanding Past Market Conditions:
Segmenting the Market: For those interested in analyzing the Airbnb market from 2015-2017, this data is crucial. It offers a clear view of how hosts and customers interacted during this period, helping to identify market segments, key growth areas, and strategies that can be applied today.

## 3. Variables and Their Distributions

For feature selection, understanding the distributions of variables is crucial. We will analyze both numerical and categorical variables.

### a. Numerical variables - Descriptive Statistics and Distribution Plots

# Display summary statistics for numerical columns

In [ ]:
df.describe()

In [ ]:
# Display summary statistics for numerical columns
df.describe()

# Plot histograms for numerical variables
df.hist(bins=20, figsize=(15, 10))
plt.tight_layout()
plt.show()

# Plot individual distributions for key numerical variables like Price
sns.histplot(df['Price'], kde=True)
plt.title('Price Distribution')
plt.show()


### b. Categorical variables - Frequency distribution

In [ ]:
# Plot the frequency of categorical variables

##CONSIDER REMOVING THIS PART
#categorical_cols = df.select_dtypes(include=['object']).columns

#for col in categorical_cols:
#    plt.figure(figsize=(10, 5))
#    df[col].value_counts().plot(kind='bar')
#    plt.title(f'{col} Distribution')
#    plt.xlabel(col)
#    plt.ylabel('Count')
#    plt.xticks(rotation=90)
#    plt.show()


# Summary and Description of Variables and Their Distributions

The dataset includes both numerical and categorical variables, each with its own distribution. Below is a summary and description of the numerical variables and their distributions, as well as a look at the categorical variables.

## Numerical Variables: Distribution Summary

#### 1. Host Response Rate:
- **Count**: 379,884
- **Mean**: 93.38%
- **Min**: 0%
- **Max**: 100%
- **Distribution**: The response rate is heavily concentrated around 100%, with most hosts being highly responsive. However, there are some hosts with 0% response rates, indicating that some hosts might not respond to queries at all.

#### 2. Host Listings Count:
- **Count**: 494,449
- **Mean**: 9.55
- **Min**: 0
- **Max**: 1,114
- **Distribution**: Most hosts have only a few listings (e.g., 1 or 2), but there are some hosts with a significantly higher number of listings, suggesting the presence of professional hosts or property managers.

#### 3. Accommodates:
- **Count**: 494,891
- **Mean**: 3.31
- **Min**: 1
- **Max**: 21
- **Distribution**: The majority of listings accommodate 1 to 4 guests, with the distribution heavily skewed towards smaller accommodations.

#### 4. Bathrooms, Bedrooms, and Beds:
- **Bathrooms**: Mean: 1.25, Max: 10
- **Bedrooms**: Mean: 1.38, Max: 96
- **Beds**: Mean: 1.93, Max: 19
- **Distribution**: These variables follow similar distributions, with most properties having between 1-3 bedrooms, bathrooms, and beds. Outliers with higher numbers of bedrooms or beds indicate larger properties, such as mansions or multi-bedroom apartments.

#### 5. Price:
- **Count**: 486,996
- **Mean**: 138
- **Min**: 0
- **Max**: 999
- **Distribution**: The price distribution is likely right-skewed, with most listings being affordable or moderately priced, and fewer listings priced extremely high (luxury listings).

#### 6. Review Scores (Accuracy, Cleanliness, Checkin, etc.):
- **Range**: All review scores (Accuracy, Cleanliness, Communication, etc.) are between 2 and 10, with most scores being around 9-10.
- **Distribution**: Review scores are heavily concentrated at the higher end (9-10), suggesting that most listings have good reviews. This might reflect self-selection, where only well-performing listings remain on the platform.

#### 7. Number of Reviews:
- **Count**: 494,952
- **Mean**: 16.74
- **Max**: 735 reviews
- **Distribution**: Most listings have fewer reviews, but a small number have a very high number of reviews, indicating popular or long-established listings.

#### 8. Reviews per Month:
- **Mean**: 1.48
- **Min**: 0.01
- **Max**: 223
- **Distribution**: There is a small number of highly active listings with many reviews per month, while the majority have far fewer reviews.

## Categorical Variables: Frequency Distribution

1. **Room Type**  
   Common categories include Entire Home/Apt, Private Room, and Shared Room.  
   **Distribution**: The majority of listings are either "Entire Home/Apt" or "Private Room," with "Shared Room" being the least frequent.

2. **Host Location**  
   This includes various cities or countries, with major cities such as New York, London, and Paris being common locations.  
   **Distribution**: The distribution is concentrated in well-known cities, with a smaller number of listings in less popular locations.

3. **Property Type**  
   Categories include Apartment, House, Condo, Loft, and others.  
   **Distribution**: Apartment and House categories dominate, while more specialized property types like Cabin or Villa are less common.

4. **Cancellation Policy**  
   Categories include Flexible, Moderate, and Strict.  
   **Distribution**: Flexible and moderate policies are more frequent, with fewer listings offering strict cancellation policies.

5. **Smart Location, Country, and Neighborhood**  
   These variables represent information about where the listing is located, including regions, cities, or countries.  
   **Distribution**: The dataset shows a concentration of listings in popular cities or regions, particularly those with high Airbnb demand.

#### Conclusion:

The numerical variables show a wide range of values and distributions, with host listings count, price, and reviews per month having both concentrated and extreme values. The categorical variables offer rich, qualitative insights into the types of listings and host locations, with a clear concentration in popular cities and property types. Understanding these distributions is essential for making informed decisions about which features to prioritize in analysis or predictive modeling.


## 4. Anomalies - Outliers 

Outliers can distort the predictive accuracy of machine learning models, so detecting them is essential. In this case, boxplot was used for this purpose.

### Detecting outliers using boxplots

In [ ]:
# Plot boxplots to identify outliers in numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

for col in numerical_cols:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=df[col])
    plt.title(f'{col} Boxplot')
    plt.show()


In [ ]:
# Calculate IQR for each numerical column
Q1 = df[numerical_cols].quantile(0.25)
Q3 = df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1

# Define outliers as values outside the range (Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers_iqr = ((df[numerical_cols] < lower_bound) | (df[numerical_cols] > upper_bound))

# Print the number of outliers for each column
# Count total outliers for each numerical column
print("\nOutliers detected using IQR method:")
print(outliers_iqr.sum())


# Description of Outliers Detected Using the IQR Method

The outlier detection using the **Interquartile Range (IQR)** method has identified a number of variables with values outside the expected range, indicating potential outliers in the dataset. Below is a description of each of the variables with detected outliers and their potential implications.

---

### 1. **Host Response Rate** (81,366 outliers):
- **Description**: The `Host Response Rate` is the percentage of queries a host responds to. A significant number of outliers in this column might indicate either hosts with **extremely low response rates** (near 0%) or those with **perfect response rates** (100%).
- **Implication**: Extreme response rates may be caused by hosts who are either inactive or highly responsive, which might not reflect typical host behavior. These outliers should be further investigated.

### 2. **Host Listings Count** and **Host Total Listings Count** (63,362 outliers):
- **Description**: These columns represent the number of listings hosted by a particular individual. The presence of outliers in these columns suggests that some hosts may have a significantly larger number of listings than others.
- **Implication**: Professional property managers or large-scale hosts may dominate this space, having many more listings than typical hosts. These outliers might need special handling if you’re analyzing listings on an individual basis.

### 3. **Accommodates** (24,433 outliers):
- **Description**: The `Accommodates` column represents the number of people a listing can host. Outliers in this column might indicate extremely large or unusually small accommodations.
- **Implication**: A small number of outliers with very high values (e.g., 20+ guests) might correspond to large homes or event spaces, while outliers with low values might correspond to listings with very limited accommodations, like small rooms.

### 4. **Bathrooms, Bedrooms, and Beds** (114,272, 18,020, and 57,328 outliers):
- **Description**: These columns represent the number of bathrooms, bedrooms, and beds in each listing. Outliers suggest that some listings have unusual configurations of these features, such as a very high number of bathrooms, bedrooms, or beds.
- **Implication**: These outliers likely correspond to larger properties, such as villas, mansions, or multi-bedroom apartments. In such cases, understanding the distribution of these outliers could help in pricing or segmentation.

### 5. **Square Feet** (393 outliers):
- **Description**: This column represents the size of the property. The outliers here are likely to be listings with extremely large or very small properties.
- **Implication**: Large outliers could indicate luxury properties, while smaller outliers might suggest atypically small living spaces. These extreme values should be reviewed, particularly if you're performing price-based analysis.

### 6. **Price** (48,906 outliers):
- **Description**: The `Price` column contains the nightly cost of renting the listing. Outliers here often indicate **luxury properties** or **erroneous data** (e.g., listings with very low or very high prices).
- **Implication**: A small number of listings might have extremely high prices, possibly due to luxury properties or data input errors. Such outliers should be handled carefully, especially if the goal is to predict prices for a typical listing.

### 7. **Weekly Price and Monthly Price** (0 and 8,648 outliers):
- **Description**: These columns represent the cost of the listing for a week or a month. A lack of values for the **Weekly Price** (0 outliers) could suggest that this feature is missing for most listings. **Monthly Price** outliers represent a few listings with very high monthly prices.
- **Implication**: The absence of weekly prices for most listings indicates that this feature is not commonly used, while the monthly price outliers might indicate properties with extreme rates or large discrepancies in pricing.

### 8. **Security Deposit and Cleaning Fee** (7,518 and 20,997 outliers):
- **Description**: These columns represent additional charges for the property. Outliers may indicate unusually high security deposits or cleaning fees for specific properties.
- **Implication**: Properties with very high security deposits or cleaning fees might be luxury or high-maintenance properties. These fees could also reflect anomalies in data reporting.

### 9. **Guests Included and Extra People** (40,629 and 34,474 outliers):
- **Description**: These columns represent the number of guests included in the base price and the cost for extra guests. Outliers in these values might indicate large accommodations with high additional charges for extra guests.
- **Implication**: Listings with a high number of extra guests or a large guest base should be examined, as they may suggest large venues or event spaces, which could influence pricing.

### 10. **Minimum Nights and Maximum Nights** (38,496 and 703 outliers):
- **Description**: These columns define the minimum and maximum stay required by the host. Outliers suggest that some listings have unusually high or low stay requirements.
- **Implication**: Properties with high minimum or maximum nights could correspond to specific event spaces or unique listings with special rules.

### 11. **Availability 30, Availability 60, Availability 90, Availability 365** (0 outliers):
- **Description**: These columns represent the availability of listings over different time frames. A value of 0 in these columns might suggest that the listing is not available for booking.
- **Implication**: These outliers could indicate listings that are either permanently unavailable or those with unavailable booking data.

### 12. **Number of Reviews** (55,580 outliers):
- **Description**: This column indicates the total number of reviews for each listing. Outliers here suggest some listings have a very high number of reviews.
- **Implication**: Listings with high numbers of reviews are often long-established or very popular. These outliers should be considered in the context of host engagement and listing activity.

### 13. **Review Scores (Rating, Accuracy, Cleanliness, Checkin, Communication, Location, Value)** (Outliers ranging from 7,518 to 11,020):
- **Description**: These columns represent various ratings given by guests. Outliers suggest that certain listings have unusually high or low scores in specific categories.
- **Implication**: Listings with outlier review scores, especially those with very low ratings, may need closer inspection. These could reflect exceptional listings with very high guest satisfaction or problematic listings that received poor reviews.

### 14. **Calculated Host Listings Count** (89,056 outliers):
- **Description**: This column represents the total number of listings a host has, calculated from their host profile. Outliers here suggest some hosts manage a very large number of listings.
- **Implication**: Professional hosts or property managers are likely represented in the outliers, and the extreme values might need to be treated separately for more accurate analysis.

### 15. **Reviews per Month** (22,630 outliers):
- **Description**: This column represents the number of reviews a listing receives on average each month. Outliers here could suggest listings that receive a very high number of reviews on a regular basis.
- **Implication**: Listings with a high number of reviews per month are likely highly popular or have been on the platform for a long time. These should be considered when assessing engagement and pricing.


### Conclusion:
The IQR method has detected many outliers across various variables in the dataset, with some columns having unusually high or low values. These outliers often correspond to listings with specific characteristics, such as luxury properties, large accommodations, or highly active hosts. It is important to handle these outliers appropriately as they might significantly impact certain modeling techniques or analyses.


## Data Cleaning 

#### 1. Removal of columns with high missing values:
We will deal with columns with high missing values by removing them from the dataframe.

In [ ]:
cols_to_drop = ["Space", "Neighborhood Overview", "Notes", "Transit", "Access", 
                "Interaction", "House Rules", "Host About", "Host Acceptance Rate",
                "Neighbourhood Group Cleansed", "Square Feet", "Weekly Price", 
                "Monthly Price", "Security Deposit", "Cleaning Fee", "License", "Jurisdiction Names", "Has Availability"]

df.drop(columns=cols_to_drop, inplace=True)


#### 1a. Removal of ID and URL columns:
We'll drop these columns as well since they won't add any predictive power to our models.

In [ ]:
df.drop(columns=['Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url', 
                 'Host Thumbnail Url', 'Host Picture Url', 'ID','Listing Url', 'Scrape ID', 'Host ID',
                'Host URL'], inplace=True)

#### 1b. Removal of other text based columns:
We dropped Name, Description, and Summary since noise would be present if they are encoded by their mean price in Mean Target Encoding. Geolocation is also removed since it leads to redundancy and could lead to overfitting due to being highly correlated to Price. 

In [ ]:
df = df.drop(columns=['Name', 'Description', 'Summary', 'Geolocation'])

#### 2. Perform Data Imputation for columns with low to moderate missing values:
We will deal with these columns by imputing median and modes or other appropriate values.

In [ ]:
# fill missing cols with appropriate values
categorical_fill_values = {
    "Host Response Time": "Unknown",
    "Host Neighbourhood": "Unknown",
    "State": df["State"].mode(dropna=True)[0] if "State" in df.columns else "Unknown",
    "Zipcode": df["Zipcode"].mode(dropna=True)[0] if "Zipcode" in df.columns else "00000",
    "Market": df["Market"].mode(dropna=True)[0] if "Market" in df.columns else "Unknown",
    #"Name": "Unnamed Listing",
    #"Summary": "No description",
    #"Description": "No description",
    "Property Type": df["Property Type"].mode(dropna=True)[0] if "Property Type" in df.columns else "Other",
    "Room Type": df["Room Type"].mode(dropna=True)[0] if "Room Type" in df.columns else "Unknown",
}

for col, value in categorical_fill_values.items():
    if col in df.columns:
        df[col] = df[col].fillna(value)

# specify the column names to fill with median values
numerical_fill_values = ["Host Response Rate", "Bathrooms", "Bedrooms", "Beds", "Price"]

for col in numerical_fill_values:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median(skipna=True))

# fill review-related numerical columns with median
review_cols = [
    "Review Scores Rating", "Review Scores Accuracy", "Review Scores Cleanliness",
    "Review Scores Checkin", "Review Scores Communication", "Review Scores Location",
    "Review Scores Value"
]

for col in review_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median(skipna=True))

# fill Reviews per Month with 0 since missing most likely means there weren't any reviews
if "Reviews per Month" in df.columns:
    df["Reviews per Month"] = df["Reviews per Month"].fillna(0)


# fill availability columns
availability_cols = ['Availability 30', 'Availability 60', 'Availability 90', 'Availability 365']
df[availability_cols] = df[availability_cols].fillna(0)

# fill remaining columns with appropriate values
df['Host Name'] = df['Host Name'].fillna("Unknown")
df['Host Since'] = df['Host Since'].fillna("Unknown")
df['Host Location'] = df['Host Location'].fillna("Unknown")
df['Street'] = df['Street'].fillna("Unknown")
df['Neighbourhood'] = df['Neighbourhood'].fillna("Unknown")
df['Neighbourhood Cleansed'] = df['Neighbourhood Cleansed'].fillna(df['Neighbourhood Cleansed'].mode()[0])
df['City'] = df['City'].fillna(df['City'].mode()[0])
df['Smart Location'] = df['Smart Location'].fillna("Unknown")
df['Country Code'] = df['Country Code'].fillna(df['Country Code'].mode()[0])
df['Country'] = df['Country'].fillna(df['Country'].mode()[0])
df['Cancellation Policy'] = df['Cancellation Policy'].fillna(df['Cancellation Policy'].mode()[0])
df['Features'] = df['Features'].fillna("None")
df['Latitude'] = df['Latitude'].fillna(df['Latitude'].median())
df['Longitude'] = df['Longitude'].fillna(df['Longitude'].median())
df['Accommodates'] = df['Accommodates'].fillna(df['Accommodates'].median())
df['Guests Included'] = df['Guests Included'].fillna(df['Guests Included'].median())
df['Extra People'] = df['Extra People'].fillna(df['Extra People'].median())
df['Minimum Nights'] = df['Minimum Nights'].fillna(df['Minimum Nights'].median())
df['Maximum Nights'] = df['Maximum Nights'].fillna(df['Maximum Nights'].median())
df['Number of Reviews'] = df['Number of Reviews'].fillna(0)
df['Host Listings Count'] = df['Host Listings Count'].fillna(0)
df['Host Total Listings Count'] = df['Host Total Listings Count'].fillna(0)
df['Calculated host listings count'] = df['Calculated host listings count'].fillna(0)
df['Last Scraped'] = df['Last Scraped'].fillna("2016-01-01")
df['Host Verifications'] = df['Host Verifications'].fillna("Unknown")  
df['Bed Type'] = df['Bed Type'].fillna(df['Bed Type'].mode()[0])
df['Amenities'] = df['Amenities'].fillna("No Amenities")
df['Calendar Updated'] = df['Calendar Updated'].fillna("Unknown")
df['Calendar last Scraped'] = df['Calendar last Scraped'].fillna("2016-01-01")
df['First Review'] = df['First Review'].fillna("2016-01-01")
df['Last Review'] = df['Last Review'].fillna("2016-01-01")
#df['Geolocation'] = df['Geolocation'].fillna("0, 0")

### Missing Data Handling and Imputation Strategy

After detecting missing data and identifying outliers in our dataset, we determined that a combination of dropping irrelevant or problematic columns and imputing values for columns with low to moderate missing data would be the most effective approach. This allowed us to retain valuable information while removing features that either contributed little to the analysis or contained excessive missing values. By dropping unnecessary columns and imputing missing values, we ensured the integrity of the dataset and minimized data loss, which could negatively impact our analysis and model predictions.

Below are the specific imputation strategies we employed, along with the columns we chose to drop and the reasoning behind these decisions:

#### Categorical Variables
For categorical columns like "Host Response Time," "Host Neighbourhood," "State," and "Market," we imputed missing values with the most common (mode) value or a placeholder like "Unknown" when appropriate. This ensures that these columns retain their categorical nature, preventing the introduction of bias or skew in our models. For example, in the case of "State" or "Zipcode," using the mode ensures that the imputed values represent the most common responses, making the dataset more representative.

#### Numerical Variables
For numerical columns such as "Host Response Rate," "Bathrooms," "Bedrooms," "Beds," and "Price," we used the median to fill missing values. The median was chosen because it is less sensitive to outliers compared to the mean, which helps preserve the integrity of the dataset. This approach is particularly suitable for numerical columns where extreme values or outliers could distort the overall distribution.

#### Review Scores
Missing values in review-related columns like "Review Scores Rating," "Review Scores Accuracy," and others were also imputed using the median, following the same rationale as for other numerical variables. This ensures that the review data is complete, enabling us to maintain consistency in evaluating the listings.

#### Reviews per Month
We imputed missing values in the "Reviews per Month" column with 0, based on the assumption that missing values likely indicate listings with no reviews. This helps ensure that the column reflects the true activity for each listing.

#### Availability Columns
For availability columns such as 'Availability 30' and 'Availability 365,' we filled missing values with 0, as a missing value here likely means no availability was recorded.

#### Other Remaining Columns
For other columns like "Host Name," "Host Since," "Street," and "Neighbourhood," we filled missing values with default placeholders such as "Unknown" or used the mode for specific columns like "Neighbourhood Cleansed" and "City." This is to ensure that we don’t lose any data and keep the dataset structured without introducing inaccuracies.

#### Geolocation
While we didn't impute "Geolocation" in this case, we would typically fill missing values with a default value (e.g., "0,0") if needed, to preserve the column for any spatial analysis, even if the actual geolocation data is unavailable.

By performing data imputation in this way, we ensured that our dataset remained comprehensive and usable for predictive modeling, analysis, and machine learning tasks. This approach allows us to handle missing data without dropping potentially valuable features, helping us build more robust models while preserving the quality of the data.


In [ ]:
# confirm that all missing values have been successfully imputed
print(df.isnull().sum().sum())

In [ ]:
# export cleaned data
df.to_csv('data/airbnb-listings_cleaned.csv', index=False)

# 3. Relationships

### a. Correlations
We converted categorical and datetime variables to numerical values with feature encoding. Then, we used Mean Target Encoding to substitute each category with the mean price of that category for all the categorical variables. In order represent the datetime variables in a numerical format, we converted them to Unix timestamps. A correlation matrix heatmap is then created to notice the connections among the various variables in the dataset and correlations are then found among the variables with the target variable Price. 

In [ ]:
categorical_cols = df.select_dtypes(include=['object']).columns
df_le = df.copy()
for col in categorical_cols:
    le = LabelEncoder()
    df_le[col] = df_le.groupby(col)['Price'].transform('mean')
datetime_column = df.select_dtypes(include=['datetime64']).columns
for col in datetime_column:
    df_le[col] = df_le[col].astype('int64') // 10**9

In [ ]:
corr_matrix = df_le.corr()
sns.heatmap(corr_matrix)

# Set the title of the plot
plt.title('Correlation Heatmap of Transformed Features with Price', fontsize=12)
plt.show()

#### 1. Explanation on Correlations among Variables with target variable Price
We found the absolute correlations among features with the target variable called Price to find the most correlated variables. Stronger connections to Price are represented by higher correlation values and this can aid in selecting features for modeling driven by data. 

In [ ]:
#Correlations of other variables with Target Variable called Price
price_corr = corr_matrix['Price'].abs().sort_values(ascending=False)
price_corr[1:31]

In [ ]:
# Encode categorical features with mean price
categorical_cols = df.select_dtypes(include=['object']).columns
df_le = df.copy()
for col in categorical_cols:
    le = LabelEncoder()
    df_le[col] = df_le.groupby(col)['Price'].transform('mean')

# Convert datetime columns to integer format (seconds since epoch)
datetime_column = df.select_dtypes(include=['datetime64']).columns
for col in datetime_column:
    df_le[col] = df_le[col].astype('int64') // 10**9

# Compute the correlation matrix
corr_matrix = df_le.corr()

# Select the correlations of other variables with the target variable "Price"
price_corr = corr_matrix['Price'].abs().sort_values(ascending=False)

# Select top 30 correlations excluding the target variable itself
top_30_corr = price_corr[1:31]

# Plot the correlation bar chart
plt.figure(figsize=(12, 8))
top_30_corr.plot(kind='barh', color='lightcoral')
plt.title('Top 30 Correlations with Price', fontsize=16)
plt.xlabel('Correlation', fontsize=12)
plt.ylabel('Variables', fontsize=12)
plt.show()


The correlation analysis of various variables with the target variable, "Price," reveals some clear trends regarding what influences property prices. The strongest correlations are observed in Amenities (0.91) and Street (0.76). Properties with more or higher-quality amenities, as well as those located on desirable streets, tend to have significantly higher prices. This suggests that both the physical features of the property and its location play major roles in determining its value.

Other location-based variables, such as Neighbourhood Cleansed (0.65), Smart Location (0.64), City (0.64), and Zipcode (0.63), also show strong positive correlations with price. This indicates that properties in well-maintained neighborhoods, areas with advanced technological features, and high-demand cities or zip codes are generally priced higher. Additionally, the Host Location (0.62) and the Calendar Last Scraped (0.61) further highlight the importance of the host’s location and the timing of the data, as these aspects appear to have a significant effect on pricing.

Moderately strong correlations are seen in variables like Market (0.60), Host Neighbourhood (0.56), and Host Name (0.53). These suggest that market conditions, the host's reputation, and the general affluence of the host’s neighborhood contribute to higher prices, although to a lesser extent than location and amenities.

In contrast, the correlations for factors such as Accommodates (0.35), Bedrooms (0.34), and Room Type (0.30), while still meaningful, are weaker. These variables indicate that the size of the property or the type of room can influence price, but not as significantly as location and amenities. The impact of Last Review (0.28), Beds (0.28), and other features like Bathrooms (0.24) and Extra People (0.23) is relatively minor, showing that these factors have a lesser effect on pricing.

In summary, the analysis highlights that location, amenities, and market conditions are the most significant drivers of price, while the size of the property, host reputation, and other features have a smaller, but still relevant, impact. The top 30 variables show varying degrees of influence, with the strongest correlations coming from factors related to location, amenities, and host-related attributes.

### b. Feature Importances
We split the dataset into training and testing sets such that 80% of the data is training and 20% is testing. We then trained a Random Forest Regressor to find feature importances and the notebook displays the top 20 most significant features from this model being trained. 

In [ ]:
#Splitting data for Feature Importances
X = df_le.drop('Price', axis=1)
y = df_le['Price']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)


In [ ]:
rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

#### 1. Explanation on Random Forest Feature Importance
Here are the feature importances for the top 20 most important variables in predicting Airbnb prices using a Random Forest Regressor. The price estimation of this model is more influenced by higher feature importances. As a result, based on these feature importances, Amenities is considered the most important feature followd by Street and Host Name. These results can help in enabling predictive performance to be better and in selecting features. 

In [ ]:
rf_im_df = pd.DataFrame({'feature': X.columns, 'importance': rf.feature_importances_}).sort_values('importance', ascending=False)
print('Random Forest Feature Importance:')
print(rf_im_df.head(20))